In [ ]:
import numpy as np

## SVRG

def svrg(itr1,, itr2, stepsize, dim):
    theta_0 = np.random.randn()
    for k in range(itr1):
        g = np.mean()
        for t in range(itr2):
            pass
        return None

## SVRG2

def svrg2(itr1,, itr2, stepsize, dim):
    return None

## CURVATURE MATCHING

def curvature_matching(f, stepsize):
    return None

## ACTION MATCHING

def action_matching(f, stepsize):
    return None